###### Юлькина система типов

Программы Julia манипулируют *значениями*, и каждое значение имеет две части: часть *типа* и часть данных. Часть type отвечает на вопрос «что это за вещь?», А часть data отличает одну вещь определенного вида от любой другой вещи такого рода.

## Part 1. Типы данных

In [ ]:
typeof(3)

В этом случае типом является `Int64`, а частью данных являются биты` ... 0011`.

В Julia типы также могут быть значениями:

In [ ]:
typeof(Int64)

In [ ]:
typeof(DataType)

На самом деле тождество `typeof (typeof (x)) === DataType` сохраняется для всех значений в Julia. `DataType` является основой всей системы. Он выполняет много заданий, которые можно определить, заглянув внутрь объекта `DataType`:

### DataType Job 1: символическое описание

Он состоит из имени (которое в основном является строкой) и вектора подкомпонентов:

In [ ]:
T = typeof(1+2im)

In [ ]:
T.name

In [ ]:
T.parameters

### DataType Job 2: Номинальная иерархия типов

In [ ]:
DataTypes формируют дерево заявленных типовых отношений («x is-a y»):

DataTypes формируют дерево заявленных типовых отношений («x is-a y»):

In [ ]:
T.super

In [ ]:
T.super.super.super.super  # `Any` это встроенная вершина иерархии.

### DataType Job 3: Описание представления

In [ ]:
T.types

In [ ]:
T.name.names

In [ ]:
T.size

In [ ]:
T.mutable   # Опишите представление `type` (vs. `immutable`)

In [ ]:
T.abstract  # было ли это объявлено с `abstract`

In [ ]:
T.ninitialized

In [ ]:
T.layout

### Определение типов структур


In [ ]:
struct Point
    x::Float64
    y::Float64
end

In [ ]:
Point(1,2)

### Абстрактное vs. Конкретное

`Abstract` типы могут иметь объявленные подтипы, в то время как конкретные типы могут иметь экземпляры. Они разделены, потому что если `X` IS-A` Y`, а `Y` задает представление, то для` X` лучше иметь такое же представление. «автомобиль - это транспортное средство» правильно, потому что «транспортное средство» - это абстрактное понятие, не связанное с какой-либо спецификой. Но если я скажу вам, что даю вам Porsche, он должен выглядеть как Porsche.

Тип `T` является конкретным, если может быть какое-то значение` x` такое, что `typeof (x) === T`. Это также иногда называют «типом листа».

In [ ]:
abstract type PointLike end

# struct Point <: PointLike

## Part 2. Параметры типа

Параметры типа могут быть указаны полностью или частично:

In [ ]:
Array{Int}

In [ ]:
[1] isa Array

In [ ]:
Array{Int,2}

Тип является конкретным (может иметь экземпляры), если
    1. Он не объявлен как `abstract`
    2. все параметры указаны

In [ ]:
[1] isa Array{Int,1}

In [ ]:
[1] isa Array{Int}

In [ ]:
[1] isa Array{Number}

In [ ]:
Int <: Number

Типы с разными *указанными* параметрами просто различны и не имеют отношения к подтипам. Это называется *инвариантность*.

### Определение типов с параметрами

In [ ]:
struct GenericPoint{T<:Real}
    x::T
    y::T
end

In [ ]:
GenericPoint(1,2)

In [ ]:
GenericPoint(1.0,2.0)

In [ ]:
GenericPoint(1,2.0)

### Типы кортежей 
(? кортеж типов)

In [ ]:
typeof((1,2.0))

Очень похоже на другие типы данных, кроме
    1. Не имеют имен полей, только индексы
    2. `T.parameters == T.types`
    3. Всегда неизменны
    4. Могут иметь любое количество полей

Эти факторы помогают сделать Tuples единственным *ковариантным* типом в Julia:

In [ ]:
Tuple{Int} <: Tuple{Number}

Тип Tuple является конкретным, если все его поля имеют типы.

Типы кортежей могут быть абстрактными относительно количества элементов. Это так называемые типы кортежей с переменным числом типов или типы vararg.

In [ ]:
Tuple{Int,Vararg{Int}}

Обратите внимание, что `Vararg` относится к хвосту типа кортежа, и как таковой не является типом самого класса. Это имеет смысл только внутри типа Tuple. Это немного неудачно. 

Второй параметр `Vararg` - это длина, которая также может быть либо неопределенной (как указано выше), либо заданной:

In [ ]:
Tuple{Int,Vararg{Int,2}}

## Part 3. Объединение типов

### Union types

Тип можно рассматривать как набор возможных значений. Тип выражает *неопределенность* относительно того, какое значение мы имеем. Вы можете делать операции над ними.

In [ ]:
Union{Int64,Float64}

In [ ]:
1 isa Union{Int64,Float64}

In [ ]:
Int64 <: Number

In [ ]:
Int64 <: Union{Int64,Float64}

In [ ]:
Union{Int,String} <: Union{Int,String,Float32}

In [ ]:
typeintersect(Union{Int,String}, Union{Int,String,Float32})

Типы Union, естественно, обобщают отсутствующие данные.

In [ ]:
data = [1.1, missing, 3.2, missing, 5.7, 0.4]

### UnionAll types


`Array{T,1} where T<:Integer`

означает «объединение всех типов вида Array {T, 1}, где T является подтипом Integer».

Это выражает неопределенность в отношении значения параметра.

Эта концепция существует во всех версиях Julia, но не имеет синтаксиса или полностью правильной поддержки в системе до следующего v0.6.0 (в настоящее время в ветке jb / subtype).

* Поскольку этот тип вводит *переменные*, его выразительная сила (вероятно) эквивалентна количественным логическим формулам.
* Требует  q-SAT решатель в компиляторе.
* По общепринятым предположениям, сложнее, чем NP-полная.

In [ ]:
# это определение в базовой библиотеке
Vector = Array{T,1} where T

Они используются для выражения «неопределенных параметров».

Они также описывают методы с «параметрами метода» (или «статическими параметрами»):

In [ ]:
func(a::Array{T,1}) where {T<:Integer} = T

In [ ]:
func([0x00])

In [ ]:
func([1.0])

#### Вопрос

В чем разница между

`Vector{Vector{T}} where T`

и

`Vector{Vector{T} where T}`?

Является ли один подтипом другого?

### Упражнение

Определите параметрический структурный тип `UnitPoint {&lt;: Real}`, который имеет поля `x` и` y` типа `T` и который имеет внутренний конструктор, который нормализует свои аргументы путем погружения их в` hypot (x, y) ` после вызова конструктора, гарантируя, что полученная точка находится на единичной окружности.